In [1]:
# %%
import pandas as pd
import torch
import torch.nn
import torch.optim
import torch.utils.data
import torchvision.datasets
import torchvision.models
import torchvision.transforms as T
from sklearn import metrics
import seaborn as sns
import time
import math
import numpy as np
import matplotlib.pyplot as plt


# %%
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)


# %%
transform = T.Compose(
    [T.Resize(224),
     T.ToTensor(),
     T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_set = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(
    train_set, batch_size=32, shuffle=True)
test_set = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(
    test_set, batch_size=32, shuffle=True)


# %%
def get_model_mobilenet_cifar10():
    model = torchvision.models.mobilenet_v2(
        weights=torchvision.models.MobileNet_V2_Weights.IMAGENET1K_V2)
    model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 10)
    model = model.to(device)
    # print(model)
    return model


# %%


def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


# %%
# Train the model
def model_training(model, criterion, optimizer, trainloader, testloader, num_epochs=10,model_name='model'):
    start = time.time()
    loss_list = []
    with torch.profiler.profile(
            schedule=torch.profiler.schedule(
                wait=1, warmup=1, active=3, repeat=2),
            on_trace_ready=torch.profiler.tensorboard_trace_handler(
                './logs/'+model_name),
            record_shapes=True,
            profile_memory=True,
            with_stack=True
    ) as prof:
      for epoch in range(num_epochs):
          train_loss = 0.0
          val_loss = 0.0
          train_acc = 0.0
          val_acc = 0.0
          for images, labels in trainloader:
              images, labels = images.to(device), labels.to(device)
              # Forward pass
              outputs = model(images)
              loss = criterion(outputs, labels)

              # Backward and optimize
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()

              train_loss += loss.item()
              train_acc += metrics.accuracy_score(labels.cpu().detach(
              ).numpy(), outputs.cpu().detach().numpy().argmax(axis=1))
              prof.step()
          # Evaluate the model on the validation set
          with torch.no_grad():
              for images, labels in testloader:
                  images, labels = images.to(device), labels.to(device)
                  outputs = model(images)
                  loss = criterion(outputs, labels)
                  val_loss += loss.item()
                  val_acc += metrics.accuracy_score(labels.cpu().detach(
                  ).numpy(), outputs.cpu().detach().numpy().argmax(axis=1))
          train_loss = train_loss/len(trainloader)
          val_loss = val_loss/len(testloader)
          train_acc = train_acc/len(trainloader)
          val_acc = val_acc/len(testloader)
          print(f'Epoch: {epoch+1} ({timeSince(start)}) \tTraining Loss: {train_loss:.3f}, \tTest Loss: {val_loss:.3f},  \tTraining acc: {train_acc:.2f}, \tTest acc: {val_acc:.2f}, ')
          loss_list.append([train_loss, val_loss, train_acc, val_acc])

      print(
          f'Training completed in {timeSince(start)} \tTraining Loss: {loss_list[-1][0]:.3f}, \tTest Loss: {loss_list[-1][1]:.3f},  \tTraining acc: {loss_list[-1][2]:.2f}, \tTest acc: {loss_list[-1][3]:.2f}, ')
      return np.array(loss_list), time.time()-start, loss_list[-1][2], loss_list[-1][3]


# %%
#sns.set(rc={'axes.facecolor': 'lightblue', 'figure.facecolor': 'lightblue'})


def confusionMatrixAndAccuracyReport(Y_test, Y_pred_probs, label):
    Y_pred = Y_pred_probs.argmax(axis=1)
    cm = metrics.confusion_matrix(Y_test, Y_pred)
    overallAccuracy = metrics.accuracy_score(Y_test, Y_pred)

    classwiseAccuracy = cm.diagonal()/cm.sum(axis=1)

    top_5_accuracy = metrics.top_k_accuracy_score(
        Y_test, Y_pred_probs, k=5, labels=np.arange(10))

    plt.figure(figsize=(10, 10))
    plt.title(
        f'Top 1 Accuracy : {overallAccuracy*100:3.2f}% | Top 5 Accuracy : {top_5_accuracy*100:3.2f}% ', size=14)
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    sns.heatmap(data=cm, annot=True, square=True,  cmap='Blues', fmt='g')

    plt.show()
    print(f'Top 1 Accuracy: {overallAccuracy*100:3.3f}%')
    print(f'Top 5 Accuracy: {top_5_accuracy*100}%')
    print(f'Classwise Accuracy Score: \n{classwiseAccuracy}')


# %%
def plot_training_graphs(loss_list):
    fig = plt.figure(figsize=(20, 7))
    plot = fig.add_subplot(1, 2, 1)
    plot.set_title("Training vs Validation loss")
    plot.plot(loss_list[:, 0], linestyle='--', label="Training Loss")
    plot.plot(loss_list[:, 1], linestyle='-', label="Validation Loss")
    plot.set_xlabel("Epoch")
    plot.set_ylabel("Loss")
    plot.legend()
    plot = fig.add_subplot(1, 2, 2)
    plot.set_title("Training vs Validation Accuracy")
    plot.plot(loss_list[:, 2], linestyle='--', label="Training Accuracy")
    plot.plot(loss_list[:, 3], linestyle='-', label="Validation Accuracy")
    plot.set_xlabel("Epoch")
    plot.set_ylabel("Accuracy")
    plot.legend()
    plt.show()



cuda:0


100%|██████████| 170498071/170498071 [00:19<00:00, 8605357.40it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [2]:
import torchvision.models
model2 = torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.IMAGENET1K_V1).to(device)
print(model2.train())

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 249MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:

optimizer = torch.optim.Adam(model2.parameters(), lr=1e-6)
criterion = torch.nn.CrossEntropyLoss()
loss_list,t,train_a,test_a = model_training(model2,criterion,optimizer,trainloader,testloader,num_epochs=10,model_name='VGG16')
plot_training_graphs(loss_list)
with torch.no_grad():
    correct, total = 0, 0
    test_loss = 0.0
    test_labels =[]
    test_output=[]
    for batch in testloader:
        x, y = batch
        x, y = x.to(device), y.to(device)
        y_hat = model2(x)
        loss = criterion(y_hat, y)
        test_loss += loss.detach().cpu().item() / len(testloader)

        correct += torch.sum(torch.argmax(y_hat, dim=1) == y).detach().cpu().item()
        total += len(x)
    print(f"Test loss: {test_loss:.2f}")
    print(f"Test accuracy: {correct / total * 100:.2f}%")
print(f'Confusion Matrix')
confusionMatrixAndAccuracyReport(y,y_hat,test_set.classes)

Epoch: 1 (14m 48s) 	Training Loss: 2.695, 	Test Loss: 1.319,  	Training acc: 0.35, 	Test acc: 0.57, 
Epoch: 2 (29m 31s) 	Training Loss: 1.018, 	Test Loss: 0.843,  	Training acc: 0.66, 	Test acc: 0.72, 
Epoch: 3 (44m 14s) 	Training Loss: 0.692, 	Test Loss: 0.648,  	Training acc: 0.77, 	Test acc: 0.78, 
